# Exercise 3: "Set" Variable-Knapsack Problem
Model the following problem using the set variable (https://docs.ocean.dwavesys.com/en/latest/docs_optimization/reference/generated/dwave.optimization.model.Model.set.html)

There are four items to choose from with the following weights and values:

- Item 1: 10 kg; \$5
- Item 2: 20 kg; \$7
- Item 3: 5 kg; \$2
- Item 4: 15 kg; \$9

The knapsack has a capacity of 30 kg. Select which items to put in the knapsack to maximize the total value of included items while staying under the 30 kg weight limit.

## Create Model

In [1]:
# required imports
from dwave.optimization import Model

# create blank model
model = Model()

# Add constants
weights = model.constant([10, 20, 5, 15])
values = model.constant([-5, -7, -2, -9])
capacity = model.constant(30)

# Add the decision variable
items = model.set(4)

# add the capacity constraint
model.add_constraint(weights[items].sum() <= capacity)

# Set the objective
model.minimize(values[items].sum())

## Solve Problem
Send the NL Model to the LeapHybridNLSampler

In [2]:
from dwave.system import LeapHybridNLSampler
sampler = LeapHybridNLSampler()
sampler.sample(model)

<Future at 0x1dd2b5e4a90 state=running>

## Check Results
Check Objective and Variables

In [3]:
with model.lock():
    print("Value of Selected Items:", -model.objective.state(0))
    index_selected_items = items.state(0)
    print("Index of Selected Items:",index_selected_items)
    
    selected_weights = [weights.state(0).tolist()[int(i)] for i in index_selected_items.tolist()]
    print("Weights of Selected Items:",selected_weights)

    selected_values = [-values.state(0).tolist()[int(i)] for i in index_selected_items.tolist()]
    print("Values of Selected Items:",selected_values)

Value of Selected Items: 16.0
Index of Selected Items: [3. 2. 0.]
Weights of Selected Items: [15.0, 5.0, 10.0]
Values of Selected Items: [9.0, 2.0, 5.0]


In [4]:
# Example of how two items out of 4 original items can be selected in python
# This is the idea behind using the set variable
import numpy as np
weights = np.array([10, 20, 5, 15])
weights[[1,2]]

array([20,  5])

# Exercise 4: "List" Variable-Basic routing Problem
Model the following problem using the set variable.

Consider a problem of selecting a route for several destinations with the cost increasing on each leg of the itinerary; for the example formulated below, one can travel through four destinations in any order, one destination per day, with the transportation cost per unit of travel doubling every subsequent day.

The figure below shows four destinations as dots labeled 0 to 3, and plots the least costly (green) and most costly (red) routes.

Reference: https://docs.ocean.dwavesys.com/en/latest/docs_optimization/intro.html#example-implicitly-constrained-symbols

In [5]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url="https://docs.ocean.dwavesys.com/en/stable/_images/best_worst_routes.png")

The code snippet below defines the cost per leg and the distances between the four destinations, with values chosen for simple illustration.

In [6]:
import numpy as np
cost_per_day = [1, 2, 4]
distance_matrix = np.asarray([
    [0, 1, np.sqrt(10), np.sqrt(34)],
    [1, 0, 2, np.sqrt(25)],
    [np.sqrt(10), 3, 0, 4],
    [np.sqrt(34), np.sqrt(25), 4, 0]])

## Create Model

In [7]:
from dwave.optimization import Model
model = Model()
# Add the constants
cost = model.constant(cost_per_day)
distances = model.constant(distance_matrix)
# Add the decision symbol
route = model.list(4) #[0,1,2,3]

# Optimize the objective
model.minimize((cost * distances[route[:-1],route[1:]]).sum())

## Solve Problem
Send the NL Model to the LeapHybridNLSampler

In [8]:
from dwave.system import LeapHybridNLSampler
sampler = LeapHybridNLSampler()
sampler.sample(model=model)

<Future at 0x1dd28bd4c50 state=running>

## Check Results
Check Objective and Variables

In [9]:
with model.lock():
    print(f"Objective = {model.objective.state(0)}")
    print(route.state(0))

Objective = 14.0
[3. 2. 1. 0.]
